# Noticias verdaderas

## Infobae

In [36]:
import requests
from bs4 import BeautifulSoup

def obtener_links(seccion="colombia", limite=25):
    """
    Extrae hasta `limite` links de noticias de Infobae en la sección indicada,
    usando directamente la clase 'story-card-ctn'.
    """
    url = f"https://www.infobae.com/{seccion}/"
    headers = {"User-Agent": "Mozilla/5.0"}
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    links = []
    for a in soup.find_all("a", class_="story-card-ctn", href=True):
        href = a["href"]

        # Normalizar: agregar dominio si es relativo
        if href.startswith("/"):
            href = "https://www.infobae.com" + href

        if href not in links:
            links.append(href)

        if len(links) >= limite:
            break

    return links

from bs4 import BeautifulSoup
import requests

def extraer_noticia(url):
    """
    Extrae el título y el cuerpo de una noticia de Infobae.
    
    Parámetros:
        url (str): URL de la noticia
    
    Retorna:
        list: [titulo, cuerpo]
    """
    # Hacer la petición
    response = requests.get(url, timeout=10)
    if response.status_code != 200:
        raise Exception(f"Error al acceder a la página: {response.status_code}")

    # Parsear HTML
    soup = BeautifulSoup(response.text, "html.parser")

    # Extraer título (h1 con clase 'article-headline')
    titulo = soup.find("h1", class_="article-headline")
    titulo = titulo.get_text(strip=True) if titulo else "No encontrado"

    # Extraer párrafos dentro del div con clase "body-article"
    cuerpo_div = soup.find("div", class_="body-article")
    cuerpo = ""
    if cuerpo_div:
        parrafos = cuerpo_div.find_all("p", class_="paragraph")
        cuerpo = " ".join([p.get_text(' ',strip=True) for p in parrafos])

    return titulo,cuerpo

import os
import re

def guardar_noticias(links, carpeta="Verdad"):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

    def limpiar_texto(texto):
        """
        Elimina frases promocionales como:
        'Ahora puede seguirnos enFacebooky en nuestroWhatsApp Channel'
        """
        # Regex que elimina ese bloque (con o sin espacios entre palabras)
        patron = r"Ahora puede seguirnos.*?(Facebook).*?(WhatsApp Channel)"
        return re.sub(patron, "", texto, flags=re.DOTALL)

    for i, link in enumerate(links, start=1):
        try:
            titulo, cuerpo = extraer_noticia(link)

            # Limpiar texto con regex
            cuerpo_limpio = limpiar_texto(cuerpo)

            # Guardar
            nombre_archivo = f"noticia_{i}.txt"
            ruta_archivo = os.path.join(carpeta, nombre_archivo)

            with open(ruta_archivo, "w", encoding="utf-8") as f:
                f.write(titulo + "\n")
                f.write(cuerpo_limpio)

            print(f"✅ Guardada: {ruta_archivo}")

        except Exception as e:
            print(f"❌ Error con {link}: {e}")

noticias = obtener_links()
guardar_noticias(noticias)

✅ Guardada: Verdad\noticia_1.txt
✅ Guardada: Verdad\noticia_2.txt
✅ Guardada: Verdad\noticia_3.txt
✅ Guardada: Verdad\noticia_4.txt
✅ Guardada: Verdad\noticia_5.txt
✅ Guardada: Verdad\noticia_6.txt
✅ Guardada: Verdad\noticia_7.txt
✅ Guardada: Verdad\noticia_8.txt
✅ Guardada: Verdad\noticia_9.txt
✅ Guardada: Verdad\noticia_10.txt
✅ Guardada: Verdad\noticia_11.txt
✅ Guardada: Verdad\noticia_12.txt
✅ Guardada: Verdad\noticia_13.txt
✅ Guardada: Verdad\noticia_14.txt
✅ Guardada: Verdad\noticia_15.txt
✅ Guardada: Verdad\noticia_16.txt
✅ Guardada: Verdad\noticia_17.txt
✅ Guardada: Verdad\noticia_18.txt
✅ Guardada: Verdad\noticia_19.txt
✅ Guardada: Verdad\noticia_20.txt
✅ Guardada: Verdad\noticia_21.txt
✅ Guardada: Verdad\noticia_22.txt
✅ Guardada: Verdad\noticia_23.txt
✅ Guardada: Verdad\noticia_24.txt
✅ Guardada: Verdad\noticia_25.txt


# Noticias Falsas

In [35]:
import requests
from bs4 import BeautifulSoup
import os

# Función para obtener los links de noticias de El Deforma
def obtener_links_deforma(url, limite=35):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Buscar todos los <a> que contengan links válidos
    links = []
    for a in soup.find_all("a", href=True):
        href = a["href"]
        if "eldeforma.com" in href and "/2025/" in href:  # solo noticias recientes
            if href not in links:  # evitar duplicados
                links.append(href)
        if len(links) >= limite:
            break
    return links

import os
import re
import requests
from bs4 import BeautifulSoup


def extraer_noticia_deforma(url):
    try:
        resp = requests.get(url, timeout=10)
        resp.encoding = "utf-8"
        soup = BeautifulSoup(resp.text, "html.parser")

        articulo = soup.find("article")
        if not articulo:
            return None, None

        # Buscar contenedor con párrafos
        contenedor = None
        for div in articulo.find_all("div"):
            if div.find("p"):
                contenedor = div
                break
        if not contenedor:
            return None, None

        # Eliminar basura
        for tag in contenedor.find_all(["blockquote", "iframe", "script", "figure"]):
            tag.decompose()
        for tag in contenedor.find_all("div", attrs={"data-testid": "tweetText"}):
            tag.decompose()

        titulo = soup.find("h1")
        titulo = titulo.get_text(strip=True) if titulo else None

        parrafos = [p.get_text(" ", strip=True) for p in contenedor.find_all("p")]
        cuerpo = " ".join(parrafos)

        # Limpieza regex
        cuerpo = re.sub(r'https?://\S+', '', cuerpo)
        cuerpo = re.sub(r'@\w+', '', cuerpo)
        cuerpo = re.sub(r'#\w+', '', cuerpo)
        cuerpo = re.sub(r'\s+', ' ', cuerpo).strip()

        if not cuerpo:
            return None, None

        return titulo, cuerpo

    except Exception as e:
        print(f"⚠️ Error en {url}: {e}")
        return None, None


def guardar_noticias_deforma(links, carpeta="Falsas", n=25):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

    contador = 0
    for url in links:
        if contador >= n:
            break

        titulo, cuerpo = extraer_noticia_deforma(url)

        if titulo and cuerpo:
            # Nombre del archivo limpio
            nombre_archivo = f"noticia_{contador+1}.txt"
            ruta = os.path.join(carpeta, nombre_archivo)

            with open(ruta, "w", encoding="utf-8") as f:
                f.write(titulo + "\n\n" + cuerpo)

            contador += 1
            print(f"✅ Guardada noticia {contador}: {titulo[:60]}...")
        else:
            print(f"❌ Saltada (sin contenido): {url}")

    print(f"\n📌 Total de noticias guardadas: {contador}")

links = obtener_links_deforma("https://eldeforma.com/")
guardar_noticias_deforma(links)



✅ Guardada noticia 1: Chilangos que compraron bolillo para prevenir tendrán que co...
✅ Guardada noticia 2: Tras éxito, Demon Slayer confirma 3 secuelas, 2 spin-offs y ...
✅ Guardada noticia 3: Presentadores de noticias en EU deberán alabar a Trump cada ...
✅ Guardada noticia 4: Inauguran la Presa Josefa Ortiz Téllez-Tirón, Soltera y Empo...
✅ Guardada noticia 5: Pozole que sobró no podrá ser congelado hasta las Posadas: S...
✅ Guardada noticia 6: 90% de los mexicanos solo quieren ver que los políticos se e...
✅ Guardada noticia 7: ¿Algo más, patrón? Elevan nivel de río para que vicepresiden...
✅ Guardada noticia 8: So, it begins: Inteligencia Artificial intentó copiarse por ...
✅ Guardada noticia 9: ¿No que muy acá? Consola de 1977 vence a ChatGPT en partida ...
❌ Saltada (sin contenido): https://eldeforma.com/2025/04/16/que-son-las-ptu/
❌ Saltada (sin contenido): https://eldeforma.com/2025/04/14/lego-leon/
❌ Saltada (sin contenido): https://eldeforma.com/2025/02/21/metodos-efectivos-

In [39]:
import os
import json

def generar_json(carpeta, archivo_salida, label):
    datos = []

    # Procesar carpeta de noticias falsas
    for archivo in os.listdir(carpeta):
        if archivo.endswith(".txt"):
            ruta = os.path.join(carpeta, archivo)
            with open(ruta, "r", encoding="utf-8") as f:
                lineas = f.readlines()
                titulo = lineas[0].strip() if lineas else "Sin título"
                cuerpo = " ".join(linea.strip() for linea in lineas[1:])
            
            datos.append({
                "label": label,
                "title": titulo,
                "body": cuerpo
            })

    # Guardar en un único archivo JSON
    with open(archivo_salida, "w", encoding="utf-8") as f:
        json.dump(datos, f, ensure_ascii=False, indent=4)

    print(f"✅ JSON generado en {archivo_salida} con {len(datos)} noticias")


generar_json("Verdad", "verdad.txt", 1)
generar_json("Falsas", 'falsas.txt', 0)

✅ JSON generado en verdad.txt con 25 noticias
✅ JSON generado en falsas.txt con 25 noticias
